## Set Libraries

In [1]:
import numpy as np
import xarray as xr
import rasterio
%matplotlib inline
from matplotlib.pyplot import *
from glob import glob
import os
import datetime
import glob
import pandas as pd
from rasterio_to_xarray import rasterio_to_xarray, xarray_to_rasterio

## Function to read in the MAIAC files into a data array

In [2]:
def maiac_file_to_da(filename):
    da = rasterio_to_xarray(filename)
    
    time_str = os.path.basename(filename)[17:-13]
    time_obj = datetime.datetime.strptime(time_str, '%Y%j%H%M')
    da.coords['time'] = time_obj
    
    return da

files = glob(r'D:\MAIAC_Data\Europe\h00v01\Projected\????\*_proj.tif')

In [3]:
folders = glob.glob(r'D:\MAIAC_Data\Europe\h00v01\Projected\????')

In [ ]:
folders

## For loop to read MAIAC files in, year by year and create NetCDF files for each month of each year.

### AOT

In [11]:
for folder in folders:
    files = glob.glob(r'{fname}\*_proj.tif'.format(fname=folder))
    list_of_das = map(maiac_file_to_da, files)
    MAIAC_AOT = xr.concat(list_of_das, 'time')
    reordered_MAIAC_AOT = MAIAC_AOT.isel(time=np.argsort(MAIAC_AOT.time))
    print('Loaded all data for folder: {folder}'.format(folder=folder))
    g = reordered_MAIAC_AOT.groupby('time.month')
    
    for month, indices in g.groups.items():
        print('Processing month: {month}'.format(month=month))
        subset = reordered_MAIAC_AOT.isel(time=indices)
        sub_ds = subset.to_dataset(name = 'data')
        filename = r'D:\MAIAC_Data\Europe\h00v01\nc_monthly\{fol_name}_{month}_AOT.nc'.format(fol_name=os.path.basename(folder), month=month)
        sub_ds.to_netcdf(filename)
        print(filename)
        
        print('Processed {fname}'.format(fname=folder))

Loaded all data for folder: D:\MAIAC_Data\Europe\h00v01\Projected\2000
Processing month: 2
D:\MAIAC_Data\Europe\h00v01\nc_monthly\2000_2_AOT.nc
Processed D:\MAIAC_Data\Europe\h00v01\Projected\2000
Processing month: 3
D:\MAIAC_Data\Europe\h00v01\nc_monthly\2000_3_AOT.nc
Processed D:\MAIAC_Data\Europe\h00v01\Projected\2000
Processing month: 4
D:\MAIAC_Data\Europe\h00v01\nc_monthly\2000_4_AOT.nc
Processed D:\MAIAC_Data\Europe\h00v01\Projected\2000
Processing month: 5
D:\MAIAC_Data\Europe\h00v01\nc_monthly\2000_5_AOT.nc
Processed D:\MAIAC_Data\Europe\h00v01\Projected\2000
Processing month: 6
D:\MAIAC_Data\Europe\h00v01\nc_monthly\2000_6_AOT.nc
Processed D:\MAIAC_Data\Europe\h00v01\Projected\2000
Processing month: 7
D:\MAIAC_Data\Europe\h00v01\nc_monthly\2000_7_AOT.nc
Processed D:\MAIAC_Data\Europe\h00v01\Projected\2000
Processing month: 8
D:\MAIAC_Data\Europe\h00v01\nc_monthly\2000_8_AOT.nc
Processed D:\MAIAC_Data\Europe\h00v01\Projected\2000
Processing month: 9
D:\MAIAC_Data\Europe\h00v01

### PM2.5

In [12]:
for folder in folders:
    files = glob.glob(r'{fname}\*_PM25.tif'.format(fname=folder))
    list_of_das = map(maiac_file_to_da, files)
    MAIAC_AOT = xr.concat(list_of_das, 'time')
    reordered_MAIAC_AOT = MAIAC_AOT.isel(time=np.argsort(MAIAC_AOT.time))
    print('Loaded all data for folder: {folder}'.format(folder=folder))
    g = reordered_MAIAC_AOT.groupby('time.month')
    
    for month, indices in g.groups.items():
        print('Processing month: {month}'.format(month=month))
        subset = reordered_MAIAC_AOT.isel(time=indices)
        sub_ds = subset.to_dataset(name = 'data')
        filename = r'D:\MAIAC_Data\Europe\h00v01\nc_monthly\{fol_name}_{month}_PM25.nc'.format(fol_name=os.path.basename(folder), month=month)
        sub_ds.to_netcdf(filename)
        print(filename)
        
        print('Processed {fname}'.format(fname=folder))

Loaded all data for folder: D:\MAIAC_Data\Europe\h00v01\Projected\2000
Processing month: 2
D:\MAIAC_Data\Europe\h00v01\nc_monthly\2000_2_PM25.nc
Processed D:\MAIAC_Data\Europe\h00v01\Projected\2000
Processing month: 3
D:\MAIAC_Data\Europe\h00v01\nc_monthly\2000_3_PM25.nc
Processed D:\MAIAC_Data\Europe\h00v01\Projected\2000
Processing month: 4
D:\MAIAC_Data\Europe\h00v01\nc_monthly\2000_4_PM25.nc
Processed D:\MAIAC_Data\Europe\h00v01\Projected\2000
Processing month: 5
D:\MAIAC_Data\Europe\h00v01\nc_monthly\2000_5_PM25.nc
Processed D:\MAIAC_Data\Europe\h00v01\Projected\2000
Processing month: 6
D:\MAIAC_Data\Europe\h00v01\nc_monthly\2000_6_PM25.nc
Processed D:\MAIAC_Data\Europe\h00v01\Projected\2000
Processing month: 7
D:\MAIAC_Data\Europe\h00v01\nc_monthly\2000_7_PM25.nc
Processed D:\MAIAC_Data\Europe\h00v01\Projected\2000
Processing month: 8
D:\MAIAC_Data\Europe\h00v01\nc_monthly\2000_8_PM25.nc
Processed D:\MAIAC_Data\Europe\h00v01\Projected\2000
Processing month: 9
D:\MAIAC_Data\Europe

In [ ]:
all_data = xr.open_mfdataset(r'D:\Annies_Dissertation\Methodology\Validation\AERONET\Test\*.nc', 
                                 chunks={'time':10}, concat_dim='time')
                                 
all_data = all_data['data']

In [13]:
all_data = xr.open_mfdataset(r'D:\MAIAC_Data\Europe\h00v01\nc_files\*_PM25.nc', 
                                 chunks={'time':10}, concat_dim='time')
                                 
all_data = all_data['data']

OSError: Unknown error

## For loop to read the MAIAC data in and extract AOT for a specific pixel (Chilbolton) and export as csv

In [ ]:
for folder in folders:
    files = glob.glob(r'{fname}\*_proj.tif'.format(fname=folder))
    list_of_das = map(maiac_file_to_da, files)
    MAIAC_AOT = xr.concat(list_of_das, 'time')
    MAIAC_AOT.attrs.clear()
    reordered_MAIAC_AOT = MAIAC_AOT.isel(time=np.argsort(MAIAC_AOT.time))
    Measurements = reordered_MAIAC_AOT.isel(x=1103, y=1027).dropna('time').to_dataframe(name='MAIAC_AOT')
    Measurements.to_csv(r'D:\Annies_Dissertation\Methodology\Validation\AERONET\Test\test.csv', mode='a')
    print('Processed {fname}'.format(fname=folder))

Measurements.to_csv(r'D:\Annies_Dissertation\Methodology\Validation\AERONET\Test\Alldata.csv')

In [ ]:
    doc = open(r'D:\Annies_Dissertation\Methodology\Validation\AERONET\Test\Alldata.csv', 'a')
    doc.write(Measurements)
    doc.close()